In [20]:
from PIL import Image
import os 
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error, r2_score
import random

import numpy as np
import matplotlib.pyplot as plt

# Pytorch library import
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# scikit-learn library import
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

# Pandas library import
import pandas as pd

import torchvision as torchvision
import torchvision.transforms as transforms # 데이터 전처리를 위해 사용하는 라이브러리

In [21]:
# device = torch.device("cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [37]:

images = '../../dataset/basic/rectangular_model/1.25/edge_distance(10-14)/random_triangle/images/'
png_files = sorted([f for f in os.listdir(images) if f.endswith('.png') and f.split('.')[0].isdigit()]) # 순서대로
sorted_files = sorted(png_files, key=lambda x: int(x.split(".")[0]))

cnt = 0
x_all = []
for f in sorted_files:
    path = os.path.join(images, f)
    x = Image.open(path)
    x_array = np.array(x)
    x_all.append(x_array)
    cnt += 1

    if cnt == 204800:
        break
    
print("random_triangle : ", cnt)

images = '../../dataset/basic/rectangular_model/1.25/edge_distance(10-14)/random_ellipse/images/'
png_files = sorted([f for f in os.listdir(images) if f.endswith('.png') and f.split('.')[0].isdigit()]) # 순서대로
sorted_files = sorted(png_files, key=lambda x: int(x.split(".")[0]))

cnt = 0
for f in sorted_files:
    path = os.path.join(images, f)
    x = Image.open(path)
    x_array = np.array(x)
    x_all.append(x_array)
    cnt += 1

    # if cnt == 100000:
    #     break
    
print("random_ellipse : ", cnt)

images = '../../dataset/basic/rectangular_model/1.25/edge_distance(10-14)/random_rectangular/images/'
png_files = sorted([f for f in os.listdir(images) if f.endswith('.png') and f.split('.')[0].isdigit()]) # 순서대로
sorted_files = sorted(png_files, key=lambda x: int(x.split(".")[0]))

cnt = 0
for f in sorted_files:
    path = os.path.join(images, f)
    x = Image.open(path)
    x_array = np.array(x)
    x_all.append(x_array)
    cnt += 1

    # if cnt == 100000:
    #     break

print("random_rectangular : ", cnt)

x_all = np.array(x_all)

print("x_all : ", x_all.shape)



random_triangle :  204800
random_ellipse :  204800
random_rectangular :  204800
x_all :  (614400, 100, 100)


In [38]:
# read csv file into a pandas dataframe
y_triangle = pd.read_csv("../../dataset/basic/rectangular_model/1.25/edge_distance(10-14)/random_triangle/dataset.csv", header=None)
y_ellipse = pd.read_csv("../../dataset/basic/rectangular_model/1.25/edge_distance(10-14)/random_ellipse/dataset.csv", header=None)
y_rectangular = pd.read_csv("../../dataset/basic/rectangular_model/1.25/edge_distance(10-14)/random_rectangular/dataset.csv", header=None)

# convert pandas dataframe to numpy array
# pos_init_world
pos_init_world_triangle = y_triangle.values[:,1:3]
pos_init_world_ellipse = y_ellipse.values[:,1:3]
pos_init_world_rectangular = y_rectangular.values[:,1:3]

# pos_tar_world
pos_tar_world_triangle = y_triangle.values[:,4:6]
pos_tar_world_ellipse = y_ellipse.values[:,4:6]
pos_tar_world_rectangular = y_rectangular.values[:,4:6]

# yaw_init_world
yaw_init_world_triangle = y_triangle.values[:,3] - 90 # change frame
yaw_init_world_triangle[yaw_init_world_triangle < 0] += 360 # change frame
yaw_init_world_triangle /= 360 # normalization

yaw_init_world_ellipse = y_ellipse.values[:,3] - 90 # change frame
yaw_init_world_ellipse[yaw_init_world_ellipse < 0] += 360 # change frame
yaw_init_world_ellipse /= 360 # normalization

yaw_init_world_rectangular = y_rectangular.values[:,3] - 90 # change frame
yaw_init_world_rectangular[yaw_init_world_rectangular < 0] += 360 # change frame
yaw_init_world_rectangular /= 360 # normalization

# label
isSuccess_polygon = y_triangle.values[:,6]
isSuccess_ellipse = y_ellipse.values[:,6]
isSuccess_rectangular = y_rectangular.values[:,6]

# pos_tar_local ref.local frame
pos_tar_local_triangle = pos_tar_world_triangle - pos_init_world_triangle
pos_tar_local_triangle[:,0] = np.cos(-np.radians(yaw_init_world_triangle*360))*pos_tar_local_triangle[:,0] - np.sin(-np.radians(yaw_init_world_triangle*360))*(-pos_tar_local_triangle[:,1])  
pos_tar_local_triangle[:,1] = np.sin(-np.radians(yaw_init_world_triangle*360))*pos_tar_local_triangle[:,0] + np.cos(-np.radians(yaw_init_world_triangle*360))*(-pos_tar_local_triangle[:,1])

pos_tar_local_ellipse = pos_tar_world_ellipse - pos_init_world_ellipse
pos_tar_local_ellipse[:,0] = np.cos(-np.radians(yaw_init_world_ellipse*360))*pos_tar_local_ellipse[:,0] - np.sin(-np.radians(yaw_init_world_ellipse*360))*(-pos_tar_local_ellipse[:,1])  
pos_tar_local_ellipse[:,1] = np.sin(-np.radians(yaw_init_world_ellipse*360))*pos_tar_local_ellipse[:,0] + np.cos(-np.radians(yaw_init_world_ellipse*360))*(-pos_tar_local_ellipse[:,1])

pos_tar_local_rectangular = pos_tar_world_rectangular - pos_init_world_rectangular
pos_tar_local_rectangular[:,0] = np.cos(-np.radians(yaw_init_world_rectangular*360))*pos_tar_local_rectangular[:,0] - np.sin(-np.radians(yaw_init_world_rectangular*360))*(-pos_tar_local_rectangular[:,1])  
pos_tar_local_rectangular[:,1] = np.sin(-np.radians(yaw_init_world_rectangular*360))*pos_tar_local_rectangular[:,0] + np.cos(-np.radians(yaw_init_world_rectangular*360))*(-pos_tar_local_rectangular[:,1])

pos_tar_local = np.concatenate((pos_tar_local_triangle, pos_tar_local_ellipse, pos_tar_local_rectangular), axis=0)
yaw_init_world = np.concatenate((yaw_init_world_triangle, yaw_init_world_ellipse, yaw_init_world_rectangular), axis=0)
isSuccess = np.concatenate((isSuccess_polygon, isSuccess_ellipse, isSuccess_rectangular), axis=0)


In [39]:
## split dataset to train, validation, test set

# x_train_all, x_test, y_train_all, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42)
x_train, x_val, pos_train, pos_val, yaw_train, yaw_val, y_train, y_val = train_test_split(x_all, pos_tar_local, yaw_init_world, isSuccess, test_size=0.2, random_state=42)

print("x_train: ",x_train.shape)
print("x_val: ",x_val.shape)
print("pos_train: ",pos_train.shape)
print("pos_val: ",pos_val.shape)
print("yaw_train: ",yaw_train.shape)
print("yaw_val: ",yaw_val.shape)
print("y_train: ",y_train.shape)
print("y_val: ",y_val.shape)


unique_train_all, count_train_all = np.unique(y_train, return_counts=True)
print("Unique values and their counts in y_train_all:", unique_train_all, count_train_all)

unique_test, count_test = np.unique(y_val, return_counts=True)
print("Unique values and their counts in y_test:", unique_test, count_test)

x_train:  (491520, 100, 100)
x_val:  (122880, 100, 100)
pos_train:  (491520, 2)
pos_val:  (122880, 2)
yaw_train:  (491520,)
yaw_val:  (122880,)
y_train:  (491520,)
y_val:  (122880,)
Unique values and their counts in y_train_all: [0. 1.] [279950 211570]
Unique values and their counts in y_test: [0. 1.] [69814 53066]


In [40]:
## image preprocessing

x_train = x_train.astype('float32')/255
x_val = x_val.astype('float32')/255
# x_test = x_test.astype('float32')/255

x_train = x_train.reshape(-1, 100, 100, 1)
x_val = x_val.reshape(-1, 100, 100, 1)
# x_test = x_test.reshape(-1, 100, 100, 1)

print(x_train.shape)
print(x_val.shape)
# print(x_test.shape)

pos_train = pos_train.reshape(-1, 2)
yaw_train = yaw_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)

pos_val = pos_val.reshape(-1, 2)
yaw_val = yaw_val.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)
# y_test = y_test.reshape(-1, 1)

print(pos_train.shape)
print(yaw_train.shape)
print(y_train.shape)

print(pos_val.shape)
print(yaw_val.shape)
print(y_val.shape)
# print(y_test.shape)

(491520, 100, 100, 1)
(122880, 100, 100, 1)
(491520, 2)
(491520, 1)
(491520, 1)
(122880, 2)
(122880, 1)
(122880, 1)


In [41]:
# 훈련 데이터 텐서 변환
x_train = torch.from_numpy(x_train).float()
pos_train = torch.from_numpy(pos_train).float()
yaw_train = torch.from_numpy(yaw_train).float()
y_train = torch.from_numpy(y_train).float()

# 테스트 데이터 텐서 변환
# x_test = torch.from_numpy(x_test).float()
# y_test = torch.from_numpy(y_test).float()
x_val = torch.from_numpy(x_val).float()
pos_val = torch.from_numpy(pos_val).float()
yaw_val = torch.from_numpy(yaw_val).float()
y_val = torch.from_numpy(y_val).float()

# 텐서로 변환한 데이터 건수 확인
print(x_train.shape)
print(pos_train.shape)
print(yaw_train.shape)
print(y_train.shape)

print(x_val.shape)
print(pos_val.shape)
print(yaw_val.shape)
print(y_val.shape)

# 설명변수와 목적변수의 텐서를 합침
train_dataset = TensorDataset(x_train, pos_train[:,0], pos_train[:,1], yaw_train, y_train)
val_dataset = TensorDataset(x_val, pos_val[:,0], pos_val[:,1], yaw_val, y_val)
# test_dataset = TensorDataset(x_test, y_test)

batch_size = 512

# dataset loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, drop_last=True)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, drop_last=True)

torch.Size([491520, 100, 100, 1])
torch.Size([491520, 2])
torch.Size([491520, 1])
torch.Size([491520, 1])
torch.Size([122880, 100, 100, 1])
torch.Size([122880, 2])
torch.Size([122880, 1])
torch.Size([122880, 1])


In [42]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        self.dropout = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(1155, 64) # 32 * 6 * 6 + 3
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        # self.fc4_list = nn.ModuleList([nn.Linear(32, 1) for _ in range(360)])  # 360개의 이진 분류 레이어 
        self.relu = nn.ReLU()
        # self.softmax = nn.Softmax(dim=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, pos_x, pos_y, yaw):
        x = self.conv1(x)
        x = self.relu(x)
        x = F.max_pool2d(x, kernel_size=2)

        x = self.conv2(x)
        x = self.relu(x)
        x = F.max_pool2d(x, kernel_size=2)

        x = self.conv3(x)
        x = self.relu(x)
        x = F.max_pool2d(x, kernel_size=2)

        x = self.conv4(x)
        x = self.relu(x)
        x = F.max_pool2d(x, kernel_size=2)

        x = self.dropout(x)

        x = self.flatten(x)

        x = torch.cat((x, pos_x, pos_y, yaw), dim=1)
        # x = torch.cat((x.unsqueeze(1), pos.unsqueeze(1), yaw.unsqueeze(1), dyaw.unsqueeze(1)), dim=1)

        x = self.fc1(x)
        x = self.relu(x)

        x = self.fc2(x)
        x = self.relu(x)

        x = self.fc3(x)
        x = self.relu(x)

        x = self.fc4(x)
        x = self.sigmoid(x)
        # x = self.softmax(x)
        # x = [self.fc4_list[i](x) for i in range(360)]  # 360개의 이진 분류 결과

        return x

# # 모델 인스턴스 생성
# model = Network()
# model.to(device)

# learning_rate = 0.001

# criterion = nn.BCEWithLogitsLoss()  # 이진 분류 손실 함수
# # criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# num_epochs = 5
# count = 0
# loss_list = []
# iteration_list = []
# accuracy_list = []

# predictions_list = []
# labels_list = []

# for epoch in range(num_epochs):
#   for images, labels in train_loader:
#     images, labels = images.to(device), labels.to(device)
#     train = Variable(images.view(100, 1, 100, 100))
#     labels = Variable(labels)
#     outputs = model(train)
#     print(outputs.shape)
#     print(labels.shape)
#     loss = criterion(outputs, labels)
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     count += 1

#     if not (count % 50):
#       total = 0
#       correct = 0
#       for images, labels in test_loader:
#         images, labels = images.to(device), labels.to(device)
#         labels_list.append(labels)
#         test = Variable(images.view(100, 1, 100, 100))
#         outputs = model(test)
#         # predictions = torch.max(outputs, 1)[1].to(device) # cross entropy 
#         predictions = (outputs > 0).float()
#         predictions_list.append(predictions)
#         correct += (predictions == labels).sum()
#         total += len(labels)

#       accuracy = correct * 100 / total
#       loss_list.append(loss.data)
#       iteration_list.append(count)
#       accuracy_list.append(accuracy)

#     if not (count%500):
#       print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))
# tensorflow
# ## build layer

# _x_train = Input(shape=(100, 100, 1))

# x = Conv2D(16, (3, 3), padding='same')(_x_train)
# x = Activation('relu')(x)
# x = MaxPooling2D((2, 2))(x)

# x = Conv2D(16, (3, 3), padding='same')(x)
# x = Activation('relu')(x)
# x = MaxPooling2D((2, 2))(x)

# x = Conv2D(32, (3, 3), padding='same')(x)
# x = Activation('relu')(x)
# x = MaxPooling2D((2, 2))(x)

# x = Conv2D(32, (3, 3), padding='same')(x)
# x = Activation('relu')(x)
# x = MaxPooling2D((2, 2))(x)

# x = Dropout(0.5)(x)

# x = Flatten()(x)
# x = Dense(64)(x)
# x = Activation('relu')(x)

# x = Dense(64)(x)
# x = Activation('relu')(x)

# x = Dense(32)(x)
# x = Activation('relu')(x)

# outputs = []
# for i in range(0, 360, 1):
#     output = Dense(2, activation='softmax', name='rotatable_area' + str(i))(x)    
#     outputs.append(output)

# model = Model(inputs=[_x_train], outputs=outputs)

# _learning_rate = 0.001  # set your desired learning rate here

# optimizer = Adam(learning_rate=_learning_rate)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# model.summary()

In [43]:
# 모델 인스턴스 생성
model = Network()
model.to(device)

learning_rate = 0.001

# criterion = nn.BCEWithLogitsLoss()  # binary cross entropy (use logit, add sigmoid)
# criterion = nn.CrossEntropyLoss() # cross entropy
criterion = nn.BCELoss() # binary cross entropy (use 0~1 probability)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

num_epochs = 1000
count = 0
threshold = 0.5
loss_list = []
iteration_list = []
accuracy_list = []

predictions_list = []
labels_list = []

# Initialize variables for Early Stopping
best_val_loss = float('inf')
patience = 5  # Number of epochs to wait before early stopping
current_patience = 0


for epoch in range(num_epochs):
  for images, pos_x, pos_y, yaw, labels in train_loader:
    images, pos_x, pos_y, yaw, labels = images.to(device), pos_x.to(device), pos_y.to(device), yaw.to(device), labels.to(device)
    # train = Variable(images.view(100, 1, 100, 100))
    # labels = Variable(labels)
    images = images.view(batch_size, 1, 100, 100)
    pos_x = pos_x.view(batch_size, 1)
    pos_y = pos_y.view(batch_size, 1)
    yaw = yaw.view(batch_size, 1)
    outputs = model(images, pos_x, pos_y, yaw)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    count += 1

    del images # tensor in gpu delete
    del pos_x # tensor in gpu delete
    del pos_y # tensor in gpu delete
    del yaw # tensor in gpu delete
    del labels # tensor in gpu delete
    torch.cuda.empty_cache()
    
    if not (count % len(train_loader)): # last iteration of each epoch
      total = 0
      correct = 0
      val_loss = 0

      model.eval()
      with torch.no_grad():

        for images, pos_x, pos_y, yaw, labels in val_loader:
          images, pos_x, pos_y, yaw, labels = images.to(device), pos_x.to(device), pos_y.to(device), yaw.to(device), labels.to(device)
          # labels_list.append(labels)
          # val = Variable(images.view(batch_size, 1, 100, 100))
          images = images.view(batch_size, 1, 100, 100)
          pos_x = pos_x.view(batch_size, 1)
          pos_y = pos_y.view(batch_size, 1)
          yaw = yaw.view(batch_size, 1)
          outputs = model(images, pos_x, pos_y, yaw)
          # predictions = torch.max(outputs, 1)[1].to(device) # cross entropy 
          # predictions = (outputs > 0).float()
          predictions = (outputs > threshold).float()
          # predictions_list.append(predictions)
          correct += (predictions == labels).sum()
          total += len(labels)

          _loss = criterion(outputs, labels)
          val_loss += _loss.item()
        
      val_loss /= len(val_loader)
      accuracy = correct * 100 / total
      # loss_list.append(loss.data)
      # iteration_list.append(count)
      # accuracy_list.append(accuracy)

      del images # tensor in gpu delete
      del pos_x # tensor in gpu delete
      del pos_y # tensor in gpu delete
      del yaw # tensor in gpu delete
      torch.cuda.empty_cache()
 
  print("Epoch: {}, Iteration: {}, Loss: {}, Val_Loss: {}, Val Accuracy: {}%".format(epoch, count, loss.data, _loss.data, accuracy))
  
  # Check if validation loss has improved
  if val_loss < best_val_loss:
      best_val_loss = val_loss
      current_patience = 0
      # Save the model when validation loss improves
      # torch.save(model.state_dict(), 'best_model.pth')
      torch.save(model, '../models/20240116(3).pth')
  else:
      current_patience += 1
      if current_patience >= patience:
          print("Early stopping triggered. Training stopped.")
          break
      
    # if not (count%500):
    #   print("Iteration: {}, Loss: {}, Val Accuracy: {}%".format(count, loss.data, accuracy))

Epoch: 0, Iteration: 960, Loss: 0.15668398141860962, Val_Loss: 0.21703335642814636, Val Accuracy: 91.55924987792969%
Epoch: 1, Iteration: 1920, Loss: 0.15406867861747742, Val_Loss: 0.2127721607685089, Val Accuracy: 91.79932403564453%
Epoch: 2, Iteration: 2880, Loss: 0.11427715420722961, Val_Loss: 0.1662048101425171, Val Accuracy: 93.25684356689453%
Epoch: 3, Iteration: 3840, Loss: 0.11333408206701279, Val_Loss: 0.14299264550209045, Val Accuracy: 94.32129669189453%
Epoch: 4, Iteration: 4800, Loss: 0.10818977653980255, Val_Loss: 0.13354557752609253, Val Accuracy: 94.84456634521484%
Epoch: 5, Iteration: 5760, Loss: 0.09526476263999939, Val_Loss: 0.13075029850006104, Val Accuracy: 95.18799591064453%
Epoch: 6, Iteration: 6720, Loss: 0.09349803626537323, Val_Loss: 0.12785139679908752, Val Accuracy: 95.31250762939453%
Epoch: 7, Iteration: 7680, Loss: 0.0869128555059433, Val_Loss: 0.12232893705368042, Val Accuracy: 95.33448028564453%
Epoch: 8, Iteration: 8640, Loss: 0.07951663434505463, Val_Lo